# Portfolio Trimming Strategy Analysis: A Complete Walkthrough

## Introduction

**What is this notebook about?**

This notebook explores a common portfolio management question: *Should I take partial profits when stocks rise significantly ("trimming"), or just hold forever?*

**The Question:**
- Many investors wonder if they should sell portions of winning positions to lock in gains
- The alternative is "buy-and-hold" - never selling, letting winners run indefinitely
- We'll test if trimming strategies historically improved returns or reduced risk

**What We'll Test:**
1. **Buy-and-Hold Baseline** - No trimming, just hold everything
2. **Trim at +50% Gain** - Sell 20% of position when it doubles
3. **Trim at +100% Gain** - Sell 20% when it triples  
4. **Trim at +150% Gain** - Sell 20% when it's up 2.5x

For each trim strategy, we'll test three ways to use the proceeds:
- **Pro-rata reinvestment** - Spread money across all holdings proportionally
- **SPY reinvestment** - Put money into S&P 500 ETF
- **Hold as cash** - Keep it in cash (0% return)

**Time Period:** 10 years (2015-2025)

**Portfolio:** Your actual current holdings, normalized by market value at the start

---

## Table of Contents
1. Setup & Data Loading
2. Portfolio Normalization
3. Strategy Definitions
4. Backtesting Implementation
5. Performance Metrics
6. Visualizations
7. Summary & Conclusions

---
## Step 1: Setup & Imports

First, we import the tools we need:
- **yfinance**: Download historical stock prices from Yahoo Finance
- **pandas**: Data manipulation and analysis
- **numpy**: Numerical operations
- **vectorbt**: Professional backtesting framework (portfolio simulation)
- **matplotlib/plotly**: Data visualization

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Backtesting framework
import vectorbt as vbt

# Visualization
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Make plots look nice
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✓ All libraries loaded successfully")
print(f"VectorBT version: {vbt.__version__}")

---
## Step 2: Define Portfolio & Configuration

Here we define:
1. **Current holdings** - Your actual portfolio tickers and share counts
2. **Backtest period** - 10 years of history (2015-2025)
3. **Strategy parameters** - Trim thresholds and percentages

**Why these holdings?** This represents your real portfolio, so results will be directly applicable to your situation.

In [ ]:
# Portfolio snapshot (current holdings)
PORTFOLIO = {
    'AAPL': 33,
    'ATXRF': 1355,  # OTC stock - may have limited history
    'BITF': 250,
    'COIN': 15,
    'IBIT': 22,     # Newer ETF - may not have 10 years of data
    'MSFT': 5,
    'MSTR': 25,
    'NET': 15,
    'NVDA': 35,
    'PLTR': 12,
    'QQQ': 25,
    'SPY': 7,
    'TSLA': 3,
    'UNH': 7,
    'VOO': 13,
    'XYZ': 12       # Unknown ticker - will likely be dropped
}

# Backtest configuration
START_DATE = '2015-01-01'
END_DATE = datetime.now().strftime('%Y-%m-%d')
INITIAL_CASH = 100000  # Starting capital (used for normalization)

# Trim strategy parameters
TRIM_THRESHOLDS = [0.50, 1.00, 1.50]  # 50%, 100%, 150% gain thresholds
TRIM_PERCENTAGE = 0.20  # Sell 20% of position when triggered

# Reinvestment modes
REINVEST_MODES = ['pro_rata', 'spy', 'cash']

print(f"Portfolio Configuration:")
print(f"  Tickers: {len(PORTFOLIO)}")
print(f"  Period: {START_DATE} to {END_DATE}")
print(f"  Initial Capital: ${INITIAL_CASH:,}")
print(f"  Trim Thresholds: {[f'+{int(t*100)}%' for t in TRIM_THRESHOLDS]}")
print(f"  Trim Size: {int(TRIM_PERCENTAGE*100)}% of position")

---
## Step 3: Download Historical Data

**What's happening here:**
1. Download 10 years of daily price data for all tickers
2. Filter out tickers with insufficient history (e.g., new IPOs)
3. Keep only tickers with complete data for fair comparison

**Why filter?** We need complete history for all stocks to backtest accurately. Newer stocks like IBIT (launched 2024) would skew results if partially included.

In [ ]:
def download_data(tickers, start_date, end_date):
    """
    Download historical price data and filter for data quality.
    
    Returns:
    - price_data: DataFrame with adjusted close prices
    - valid_tickers: List of tickers with complete history
    - dropped_tickers: Dictionary of dropped tickers and reasons
    """
    print(f"\n📊 Downloading data for {len(tickers)} tickers...")
    print(f"Period: {start_date} to {end_date}\n")
    
    # Download all tickers at once (faster than one-by-one)
    data = yf.download(
        tickers=list(tickers),
        start=start_date,
        end=end_date,
        progress=True,
        auto_adjust=True  # Use adjusted prices (accounts for splits/dividends)
    )['Close']
    
    # Handle single vs multiple tickers (yfinance returns different structures)
    if isinstance(data, pd.Series):
        data = data.to_frame()
    
    # Calculate data quality metrics
    total_days = len(data)
    min_required_days = int(total_days * 0.95)  # Require 95% data availability
    
    valid_tickers = []
    dropped_tickers = {}
    
    for ticker in data.columns:
        non_null = data[ticker].notna().sum()
        coverage = non_null / total_days
        
        if non_null < min_required_days:
            dropped_tickers[ticker] = f"Insufficient data: {coverage:.1%} coverage"
        else:
            valid_tickers.append(ticker)
    
    # Keep only valid tickers
    price_data = data[valid_tickers].copy()
    
    # Forward fill missing data (e.g., holidays)
    price_data = price_data.fillna(method='ffill')
    
    # Summary
    print(f"\n✓ Data Download Complete")
    print(f"  Valid tickers: {len(valid_tickers)}")
    print(f"  Dropped tickers: {len(dropped_tickers)}")
    print(f"  Trading days: {len(price_data):,}")
    
    if dropped_tickers:
        print(f"\n⚠️  Dropped Tickers:")
        for ticker, reason in dropped_tickers.items():
            print(f"  {ticker}: {reason}")
    
    return price_data, valid_tickers, dropped_tickers

# Execute download
price_data, valid_tickers, dropped_tickers = download_data(
    tickers=list(PORTFOLIO.keys()),
    start_date=START_DATE,
    end_date=END_DATE
)

# Display sample of data
print("\n📈 Sample Data (first 5 days):")
print(price_data.head())

---
## Step 4: Normalize Portfolio by Market Value

**The Problem:** Your current portfolio has varying position sizes (33 AAPL shares vs 1355 ATXRF shares). We need to convert this to dollar values at the start of our backtest.

**The Solution:**
1. Calculate market value of each position at backtest start (2015 prices)
2. Convert share counts to portfolio weights (% of total value)
3. Scale weights so total portfolio = $100,000 starting capital

**Why?** This lets us compare strategies fairly - all start with the same total value, just allocated differently based on your actual position sizes.

In [ ]:
def normalize_portfolio(portfolio_dict, price_data, initial_cash):
    """
    Convert share quantities to normalized portfolio weights.
    
    Steps:
    1. Get first-day price for each ticker
    2. Calculate market value = shares × price
    3. Calculate weights = market_value / total_value
    4. Scale to initial_cash
    """
    # Filter portfolio to only valid tickers
    filtered_portfolio = {k: v for k, v in portfolio_dict.items() if k in price_data.columns}
    
    # Get first-day prices (start of backtest)
    first_prices = price_data.iloc[0]
    
    # Calculate market values
    market_values = {}
    for ticker, shares in filtered_portfolio.items():
        market_values[ticker] = shares * first_prices[ticker]
    
    total_value = sum(market_values.values())
    
    # Calculate weights (as fraction of total)
    weights = {ticker: value / total_value for ticker, value in market_values.items()}
    
    # Calculate initial shares based on normalized weights
    initial_shares = {}
    for ticker, weight in weights.items():
        dollar_allocation = initial_cash * weight
        initial_shares[ticker] = dollar_allocation / first_prices[ticker]
    
    # Create summary DataFrame
    summary = pd.DataFrame({
        'Current_Shares': [filtered_portfolio[t] for t in weights.keys()],
        'Start_Price': [first_prices[t] for t in weights.keys()],
        'Market_Value': [market_values[t] for t in weights.keys()],
        'Weight': [weights[t] for t in weights.keys()],
        'Normalized_Shares': [initial_shares[t] for t in weights.keys()]
    }, index=weights.keys())
    
    summary = summary.sort_values('Weight', ascending=False)
    
    return weights, initial_shares, summary

# Execute normalization
weights, initial_shares, portfolio_summary = normalize_portfolio(
    portfolio_dict=PORTFOLIO,
    price_data=price_data,
    initial_cash=INITIAL_CASH
)

print("\n💼 Normalized Portfolio Breakdown:")
print("="*80)
print(portfolio_summary.to_string())
print("="*80)
print(f"\nTotal Weight: {portfolio_summary['Weight'].sum():.1%}")
print(f"Total Value: ${portfolio_summary['Market_Value'].sum():,.2f}")
print(f"\nLargest Position: {portfolio_summary.index[0]} ({portfolio_summary.iloc[0]['Weight']:.1%})")
print(f"Smallest Position: {portfolio_summary.index[-1]} ({portfolio_summary.iloc[-1]['Weight']:.1%})")

---
## Step 5: Implement Buy-and-Hold Baseline

**What is Buy-and-Hold?**
The simplest strategy: Buy stocks at the start, never sell, hold until the end.

**Why start here?**
This is our benchmark. All trim strategies will be compared against this baseline. If trimming doesn't beat buy-and-hold, it's not worth the complexity.

**Implementation:**
We use VectorBT's Portfolio simulation, which handles:
- Position sizing
- Fee calculation
- Performance tracking
- Automatic metric calculation

In [ ]:
def run_buy_and_hold(price_data, initial_shares, initial_cash):
    """
    Simulate buy-and-hold strategy.
    
    Logic:
    - Buy initial positions on day 1
    - Hold forever (no trades after initial purchase)
    - Track portfolio value over time
    """
    print("\n📈 Running Buy-and-Hold Baseline...")
    
    # Create position sizing array (shares to hold each day)
    # Shape: (num_days, num_tickers)
    target_shares = pd.DataFrame(
        data=0.0,
        index=price_data.index,
        columns=price_data.columns
    )
    
    # Set initial positions (day 1) and hold
    for ticker, shares in initial_shares.items():
        target_shares.loc[:, ticker] = shares
    
    # Run portfolio simulation
    portfolio = vbt.Portfolio.from_orders(
        close=price_data,
        size=target_shares,
        size_type='targetamount',  # Hold exact share count
        init_cash=initial_cash,
        fees=0.001,  # 0.1% trading fees (realistic for retail)
        freq='D',
        group_by=True  # Treat as single portfolio
    )
    
    # Extract key metrics
    stats = {
        'final_value': portfolio.final_value,
        'total_return': portfolio.total_return,
        'cagr': portfolio.annualized_return,
        'sharpe_ratio': portfolio.sharpe_ratio,
        'sortino_ratio': portfolio.sortino_ratio,
        'max_drawdown': portfolio.max_drawdown,
        'volatility': portfolio.annualized_volatility,
        'num_trades': portfolio.trades.count()
    }
    
    print(f"  ✓ Final Value: ${stats['final_value']:,.2f}")
    print(f"  ✓ Total Return: {stats['total_return']:.1%}")
    print(f"  ✓ CAGR: {stats['cagr']:.1%}")
    print(f"  ✓ Sharpe Ratio: {stats['sharpe_ratio']:.2f}")
    print(f"  ✓ Max Drawdown: {stats['max_drawdown']:.1%}")
    
    return portfolio, stats

# Execute buy-and-hold
bh_portfolio, bh_stats = run_buy_and_hold(
    price_data=price_data,
    initial_shares=initial_shares,
    initial_cash=INITIAL_CASH
)

---
## Step 6: Implement Trimming Strategies

**How Trimming Works:**

1. **Track each position's gain** from its cost basis (original purchase price)
2. **Trigger a trim** when position reaches threshold (e.g., +50% gain)
3. **Sell 20%** of the position at market price
4. **Reinvest proceeds** according to chosen mode:
   - **Pro-rata**: Spread cash across all positions proportionally
   - **SPY**: Buy S&P 500 index (safer, diversified)
   - **Cash**: Hold in cash (0% return, maximum safety)

**Key Detail:** After a trim, the cost basis stays at the original level. This means the same position can trigger multiple trims as it continues rising (e.g., trim at +50%, then again at +100%, etc.).

**Why test different reinvestment modes?**
- Pro-rata: Maximizes exposure to your original picks (higher risk/reward)
- SPY: Balances growth with diversification
- Cash: Most conservative, locks in gains completely

In [ ]:
class TrimStrategy:
    """
    Implements a position trimming strategy with configurable reinvestment.
    
    Parameters:
    - threshold: Gain level to trigger trim (e.g., 0.50 for +50%)
    - trim_pct: Percentage of position to sell (e.g., 0.20 for 20%)
    - reinvest_mode: 'pro_rata', 'spy', or 'cash'
    """
    
    def __init__(self, price_data, initial_shares, threshold, trim_pct, reinvest_mode):
        self.price_data = price_data
        self.initial_shares = initial_shares
        self.threshold = threshold
        self.trim_pct = trim_pct
        self.reinvest_mode = reinvest_mode
        
        # Track cost basis for each position (initialized at start prices)
        self.cost_basis = price_data.iloc[0].copy()
        
        # Track current holdings
        self.holdings = pd.Series(initial_shares)
        
        # Track cash from trims
        self.cash = 0.0
        
        # Track trim events for analysis
        self.trim_log = []
    
    def check_and_execute_trims(self, date, prices):
        """
        Check all positions for trim triggers and execute if needed.
        
        Returns: Cash generated from trims (if any)
        """
        trim_proceeds = 0.0
        
        for ticker in self.holdings.index:
            if self.holdings[ticker] == 0:
                continue
            
            current_price = prices[ticker]
            cost = self.cost_basis[ticker]
            
            # Calculate gain from cost basis
            gain = (current_price - cost) / cost
            
            # Check if trim threshold reached
            if gain >= self.threshold:
                # Calculate shares to sell
                shares_to_sell = self.holdings[ticker] * self.trim_pct
                proceeds = shares_to_sell * current_price
                
                # Execute trim
                self.holdings[ticker] -= shares_to_sell
                trim_proceeds += proceeds
                
                # Log the trim
                self.trim_log.append({
                    'date': date,
                    'ticker': ticker,
                    'gain_pct': gain,
                    'shares_sold': shares_to_sell,
                    'proceeds': proceeds,
                    'price': current_price
                })
        
        return trim_proceeds
    
    def reinvest_proceeds(self, proceeds, prices):
        """
        Reinvest trim proceeds according to selected mode.
        """
        if proceeds == 0:
            return
        
        if self.reinvest_mode == 'cash':
            # Hold as cash (do nothing)
            self.cash += proceeds
        
        elif self.reinvest_mode == 'spy':
            # Buy SPY with all proceeds
            if 'SPY' in self.holdings.index:
                spy_shares = proceeds / prices['SPY']
                self.holdings['SPY'] += spy_shares
        
        elif self.reinvest_mode == 'pro_rata':
            # Distribute proportionally to current holdings
            total_value = sum(self.holdings[t] * prices[t] for t in self.holdings.index)
            
            for ticker in self.holdings.index:
                position_value = self.holdings[ticker] * prices[ticker]
                weight = position_value / total_value if total_value > 0 else 0
                
                shares_to_buy = (proceeds * weight) / prices[ticker]
                self.holdings[ticker] += shares_to_buy
    
    def run_backtest(self):
        """
        Execute full backtest with daily trim checks and reinvestment.
        """
        # Create DataFrame to track holdings over time
        holdings_history = pd.DataFrame(
            index=self.price_data.index,
            columns=self.price_data.columns,
            data=0.0
        )
        
        # Set initial holdings
        holdings_history.iloc[0] = self.holdings
        
        # Simulate day-by-day
        for i in range(1, len(self.price_data)):
            date = self.price_data.index[i]
            prices = self.price_data.iloc[i]
            
            # Check for trim triggers
            proceeds = self.check_and_execute_trims(date, prices)
            
            # Reinvest if any trims occurred
            if proceeds > 0:
                self.reinvest_proceeds(proceeds, prices)
            
            # Record holdings
            holdings_history.iloc[i] = self.holdings
        
        return holdings_history

print("✓ TrimStrategy class defined")
print("  Ready to backtest trimming strategies with multiple reinvestment modes")

---
## Step 7: Run All Strategy Variations

Now we test all combinations:
- 3 trim thresholds (50%, 100%, 150%)
- 3 reinvestment modes (pro-rata, SPY, cash)
- = 9 trim strategies + 1 buy-and-hold baseline = **10 total strategies**

For each, we'll track:
- Final portfolio value
- Annual return (CAGR)
- Risk metrics (Sharpe, Sortino, drawdown)
- Number of trims executed

In [ ]:
# Storage for all results
all_results = {}
all_portfolios = {}

print("\n" + "="*80)
print("BACKTESTING ALL STRATEGIES")
print("="*80)

# Store buy-and-hold baseline
all_results['Buy-and-Hold'] = bh_stats
all_portfolios['Buy-and-Hold'] = bh_portfolio

# Run all trim strategy combinations
for threshold in TRIM_THRESHOLDS:
    for mode in REINVEST_MODES:
        strategy_name = f"Trim@+{int(threshold*100)}% ({mode})"
        
        print(f"\n🔄 Running: {strategy_name}")
        
        # Initialize strategy
        strategy = TrimStrategy(
            price_data=price_data,
            initial_shares=initial_shares,
            threshold=threshold,
            trim_pct=TRIM_PERCENTAGE,
            reinvest_mode=mode
        )
        
        # Run backtest
        holdings_history = strategy.run_backtest()
        
        # Create VectorBT portfolio for metric calculation
        portfolio = vbt.Portfolio.from_orders(
            close=price_data,
            size=holdings_history,
            size_type='targetamount',
            init_cash=INITIAL_CASH,
            fees=0.001,
            freq='D',
            group_by=True
        )
        
        # Calculate metrics
        stats = {
            'final_value': portfolio.final_value,
            'total_return': portfolio.total_return,
            'cagr': portfolio.annualized_return,
            'sharpe_ratio': portfolio.sharpe_ratio,
            'sortino_ratio': portfolio.sortino_ratio,
            'max_drawdown': portfolio.max_drawdown,
            'volatility': portfolio.annualized_volatility,
            'num_trades': len(strategy.trim_log),
            'cash_held': strategy.cash
        }
        
        # Store results
        all_results[strategy_name] = stats
        all_portfolios[strategy_name] = portfolio
        
        # Print summary
        print(f"  Final Value: ${stats['final_value']:,.2f}")
        print(f"  CAGR: {stats['cagr']:.1%}")
        print(f"  Sharpe: {stats['sharpe_ratio']:.2f}")
        print(f"  Trims Executed: {stats['num_trades']}")
        if mode == 'cash':
            print(f"  Cash Holdings: ${stats['cash_held']:,.2f}")

print("\n" + "="*80)
print("✓ ALL BACKTESTS COMPLETE")
print("="*80)

---
## Step 8: Performance Comparison Table

Let's create a comprehensive comparison table showing all metrics side-by-side.

**How to read this table:**
- **Final Value**: Higher is better (more money)
- **CAGR**: Higher is better (faster growth rate)
- **Sharpe Ratio**: Higher is better (better risk-adjusted returns, >1.0 is good)
- **Sortino Ratio**: Higher is better (like Sharpe, but only penalizes downside volatility)
- **Max Drawdown**: Lower is better (smaller maximum loss from peak)
- **Volatility**: Lower is better (more stable, less stressful)

In [ ]:
# Create comparison DataFrame
comparison_df = pd.DataFrame(all_results).T

# Sort by final value (best to worst)
comparison_df = comparison_df.sort_values('final_value', ascending=False)

# Format for display
display_df = comparison_df.copy()
display_df['final_value'] = display_df['final_value'].apply(lambda x: f"${x:,.0f}")
display_df['total_return'] = display_df['total_return'].apply(lambda x: f"{x:.1%}")
display_df['cagr'] = display_df['cagr'].apply(lambda x: f"{x:.1%}")
display_df['sharpe_ratio'] = display_df['sharpe_ratio'].apply(lambda x: f"{x:.2f}")
display_df['sortino_ratio'] = display_df['sortino_ratio'].apply(lambda x: f"{x:.2f}")
display_df['max_drawdown'] = display_df['max_drawdown'].apply(lambda x: f"{x:.1%}")
display_df['volatility'] = display_df['volatility'].apply(lambda x: f"{x:.1%}")

# Rename columns for readability
display_df.columns = [
    'Final Value', 'Total Return', 'CAGR', 'Sharpe', 'Sortino',
    'Max Drawdown', 'Volatility', 'Num Trims', 'Cash Holdings'
]

print("\n" + "="*120)
print("STRATEGY PERFORMANCE COMPARISON")
print("="*120)
print(display_df.to_string())
print("="*120)

# Highlight best strategy
best_strategy = comparison_df.index[0]
print(f"\n🏆 Best Strategy (by final value): {best_strategy}")
print(f"   Final Value: ${comparison_df.loc[best_strategy, 'final_value']:,.2f}")
print(f"   CAGR: {comparison_df.loc[best_strategy, 'cagr']:.2%}")
print(f"   Sharpe: {comparison_df.loc[best_strategy, 'sharpe_ratio']:.2f}")

---
## Step 9: Visualizations

### 9.1 Portfolio Value Over Time

This chart shows how each strategy's portfolio value evolved over the 10-year period.
- **Higher line = more money**
- **Smoother line = less volatile (potentially less stressful)**
- **Steeper slope = faster growth**

In [ ]:
# Create interactive portfolio value chart
fig = go.Figure()

# Plot each strategy
colors = {
    'Buy-and-Hold': '#000000',  # Black
    'pro_rata': '#FF6B6B',       # Red
    'spy': '#4ECDC4',            # Teal
    'cash': '#95E1D3'            # Light green
}

for strategy_name, portfolio in all_portfolios.items():
    # Determine color
    if strategy_name == 'Buy-and-Hold':
        color = colors['Buy-and-Hold']
        width = 3
        dash = 'solid'
    else:
        # Extract reinvest mode from name
        mode = strategy_name.split('(')[1].split(')')[0]
        color = colors.get(mode, '#999999')
        width = 2
        dash = 'solid'
    
    fig.add_trace(go.Scatter(
        x=price_data.index,
        y=portfolio.value(),
        mode='lines',
        name=strategy_name,
        line=dict(color=color, width=width, dash=dash),
        hovertemplate='%{y:$,.0f}<extra></extra>'
    ))

fig.update_layout(
    title='Portfolio Value Over Time (All Strategies)',
    xaxis_title='Date',
    yaxis_title='Portfolio Value ($)',
    hovermode='x unified',
    height=600,
    showlegend=True,
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)

fig.show()

print("📊 Portfolio value chart displayed above")

### 9.2 Drawdown Comparison

**What is a drawdown?**
The peak-to-trough decline during a specific period. It measures how much you would have lost if you bought at the worst time.

**Why it matters:**
Drawdowns show how painful the journey was, not just the destination. A strategy with high returns but deep drawdowns is psychologically harder to stick with.

**How to read:**
- 0% = At all-time high (no drawdown)
- -20% = Down 20% from peak
- Shallower drawdowns = easier to hold through volatility

In [ ]:
# Create drawdown comparison chart
fig = go.Figure()

for strategy_name, portfolio in all_portfolios.items():
    # Determine color (same as above)
    if strategy_name == 'Buy-and-Hold':
        color = colors['Buy-and-Hold']
        width = 3
    else:
        mode = strategy_name.split('(')[1].split(')')[0]
        color = colors.get(mode, '#999999')
        width = 2
    
    fig.add_trace(go.Scatter(
        x=price_data.index,
        y=portfolio.drawdowns() * 100,  # Convert to percentage
        mode='lines',
        name=strategy_name,
        line=dict(color=color, width=width),
        fill='tozeroy',
        hovertemplate='%{y:.1f}%<extra></extra>'
    ))

fig.update_layout(
    title='Drawdown Over Time (All Strategies)',
    xaxis_title='Date',
    yaxis_title='Drawdown (%)',
    hovermode='x unified',
    height=600,
    showlegend=True,
    yaxis=dict(range=[-100, 0])  # Drawdowns are always negative
)

fig.show()

print("📉 Drawdown chart displayed above")

### 9.3 Return vs Risk Scatter Plot

**The Efficient Frontier Concept:**
Ideally, we want high returns with low risk. This chart plots all strategies on a return vs. risk graph.

**How to read:**
- **X-axis (Volatility)**: Risk - lower is better
- **Y-axis (CAGR)**: Return - higher is better
- **Top-left corner is ideal**: High returns, low risk
- **Bubble size**: Sharpe ratio (bigger = better risk-adjusted returns)

In [ ]:
# Create risk-return scatter plot
fig = go.Figure()

# Extract data for plotting
volatilities = []
cagrs = []
sharpes = []
names = []
marker_colors = []

for strategy_name, stats in all_results.items():
    volatilities.append(stats['volatility'] * 100)  # Convert to percentage
    cagrs.append(stats['cagr'] * 100)
    sharpes.append(max(stats['sharpe_ratio'], 0))  # Ensure non-negative for sizing
    names.append(strategy_name)
    
    # Color by strategy type
    if strategy_name == 'Buy-and-Hold':
        marker_colors.append('#000000')
    else:
        mode = strategy_name.split('(')[1].split(')')[0]
        marker_colors.append(colors.get(mode, '#999999'))

fig.add_trace(go.Scatter(
    x=volatilities,
    y=cagrs,
    mode='markers+text',
    marker=dict(
        size=[s * 20 for s in sharpes],  # Scale Sharpe for visibility
        color=marker_colors,
        opacity=0.7,
        line=dict(color='white', width=2)
    ),
    text=names,
    textposition='top center',
    textfont=dict(size=8),
    hovertemplate='<b>%{text}</b><br>Volatility: %{x:.1f}%<br>CAGR: %{y:.1f}%<extra></extra>'
))

fig.update_layout(
    title='Risk vs Return (All Strategies)<br><sub>Bubble size = Sharpe Ratio</sub>',
    xaxis_title='Volatility (Risk) - Lower is Better →',
    yaxis_title='CAGR (Return) - Higher is Better ↑',
    height=700,
    showlegend=False,
    hovermode='closest'
)

# Add quadrant lines at median values
median_vol = np.median(volatilities)
median_cagr = np.median(cagrs)

fig.add_hline(y=median_cagr, line_dash="dash", line_color="gray", opacity=0.3)
fig.add_vline(x=median_vol, line_dash="dash", line_color="gray", opacity=0.3)

# Annotate ideal quadrant
fig.add_annotation(
    x=min(volatilities),
    y=max(cagrs),
    text="IDEAL<br>(High Return, Low Risk)",
    showarrow=False,
    font=dict(size=12, color="green"),
    opacity=0.5
)

fig.show()

print("📊 Risk-return scatter plot displayed above")

### 9.4 Performance Metrics Heatmap

A color-coded heatmap showing all metrics at a glance.
- **Green = Better**
- **Red = Worse**

In [ ]:
# Create normalized comparison for heatmap
heatmap_df = comparison_df[[
    'final_value', 'cagr', 'sharpe_ratio', 'sortino_ratio', 'max_drawdown', 'volatility'
]].copy()

# Normalize each metric to 0-100 scale
# For metrics where higher is better
for col in ['final_value', 'cagr', 'sharpe_ratio', 'sortino_ratio']:
    min_val = heatmap_df[col].min()
    max_val = heatmap_df[col].max()
    heatmap_df[col] = ((heatmap_df[col] - min_val) / (max_val - min_val)) * 100

# For metrics where lower is better (invert)
for col in ['max_drawdown', 'volatility']:
    min_val = heatmap_df[col].min()
    max_val = heatmap_df[col].max()
    heatmap_df[col] = ((max_val - heatmap_df[col]) / (max_val - min_val)) * 100

# Rename for display
heatmap_df.columns = ['Final Value', 'CAGR', 'Sharpe', 'Sortino', 'Drawdown', 'Volatility']

# Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_df.values,
    x=heatmap_df.columns,
    y=heatmap_df.index,
    colorscale='RdYlGn',  # Red-Yellow-Green
    text=np.round(heatmap_df.values, 1),
    texttemplate='%{text}',
    textfont={"size": 10},
    colorbar=dict(title="Score<br>(0-100)")
))

fig.update_layout(
    title='Performance Metrics Heatmap (Normalized 0-100)<br><sub>Green = Better, Red = Worse</sub>',
    height=600,
    xaxis_title='Metric',
    yaxis_title='Strategy'
)

fig.show()

print("🔥 Performance heatmap displayed above")

---
## Step 10: Detailed Trim Analysis

Let's look at when and how often trims were executed.
This helps understand if the thresholds were realistic (too many trims = too sensitive, too few = not useful).

In [ ]:
# Analyze trim frequency by strategy
trim_summary = pd.DataFrame({
    'Strategy': list(all_results.keys()),
    'Total Trims': [all_results[s]['num_trades'] for s in all_results.keys()]
})

trim_summary = trim_summary[trim_summary['Total Trims'] > 0].sort_values('Total Trims', ascending=False)

print("\n" + "="*80)
print("TRIM EXECUTION SUMMARY")
print("="*80)
print(trim_summary.to_string(index=False))
print("="*80)

# Calculate average trims per year
years = (pd.to_datetime(END_DATE) - pd.to_datetime(START_DATE)).days / 365.25
print(f"\nBacktest Period: {years:.1f} years")
print("\nAverage Trims Per Year:")
for _, row in trim_summary.iterrows():
    trims_per_year = row['Total Trims'] / years
    print(f"  {row['Strategy']}: {trims_per_year:.1f} trims/year")

---
## Step 11: Summary & Conclusions

### Key Findings

Let's interpret the results in plain language:

In [ ]:
# Generate automated insights
print("\n" + "="*80)
print("EXECUTIVE SUMMARY")
print("="*80)

# 1. Best overall strategy
best_final_value = comparison_df['final_value'].idxmax()
best_sharpe = comparison_df['sharpe_ratio'].idxmax()
best_cagr = comparison_df['cagr'].idxmax()
lowest_drawdown = comparison_df['max_drawdown'].idxmin()

print("\n1️⃣  WINNER BY DIFFERENT CRITERIA:\n")
print(f"   💰 Highest Final Value: {best_final_value}")
print(f"      ${comparison_df.loc[best_final_value, 'final_value']:,.0f}")
print(f"\n   📈 Highest CAGR: {best_cagr}")
print(f"      {comparison_df.loc[best_cagr, 'cagr']:.2%} per year")
print(f"\n   ⚖️  Best Risk-Adjusted (Sharpe): {best_sharpe}")
print(f"      Sharpe Ratio: {comparison_df.loc[best_sharpe, 'sharpe_ratio']:.2f}")
print(f"\n   🛡️  Smallest Drawdown: {lowest_drawdown}")
print(f"      Max Drawdown: {comparison_df.loc[lowest_drawdown, 'max_drawdown']:.1%}")

# 2. Buy-and-hold comparison
bh_value = comparison_df.loc['Buy-and-Hold', 'final_value']
print("\n\n2️⃣  DID TRIMMING BEAT BUY-AND-HOLD?\n")

better_than_bh = comparison_df[comparison_df['final_value'] > bh_value]
worse_than_bh = comparison_df[comparison_df['final_value'] < bh_value]

print(f"   Buy-and-Hold Final Value: ${bh_value:,.0f}")
print(f"\n   Strategies that outperformed: {len(better_than_bh) - 1}")  # -1 to exclude BH itself
print(f"   Strategies that underperformed: {len(worse_than_bh)}")

if len(better_than_bh) > 1:
    print("\n   ✅ YES - Some trim strategies beat buy-and-hold!")
    print("\n   Top performers vs Buy-and-Hold:")
    for strategy in better_than_bh.index[:3]:  # Top 3
        if strategy != 'Buy-and-Hold':
            outperformance = (better_than_bh.loc[strategy, 'final_value'] - bh_value) / bh_value
            print(f"      {strategy}: +{outperformance:.1%}")
else:
    print("\n   ❌ NO - Buy-and-hold beat all trim strategies")

# 3. Reinvestment mode analysis
print("\n\n3️⃣  BEST REINVESTMENT MODE:\n")

# Average performance by reinvestment mode
mode_performance = {}
for mode in REINVEST_MODES:
    mode_strategies = [s for s in all_results.keys() if f'({mode})' in s]
    if mode_strategies:
        avg_cagr = np.mean([all_results[s]['cagr'] for s in mode_strategies])
        avg_sharpe = np.mean([all_results[s]['sharpe_ratio'] for s in mode_strategies])
        mode_performance[mode] = {'cagr': avg_cagr, 'sharpe': avg_sharpe}

for mode, perf in sorted(mode_performance.items(), key=lambda x: x[1]['cagr'], reverse=True):
    print(f"   {mode.upper()}:")
    print(f"      Avg CAGR: {perf['cagr']:.2%}")
    print(f"      Avg Sharpe: {perf['sharpe']:.2f}")

# 4. Risk analysis
print("\n\n4️⃣  RISK REDUCTION:\n")

bh_drawdown = comparison_df.loc['Buy-and-Hold', 'max_drawdown']
bh_vol = comparison_df.loc['Buy-and-Hold', 'volatility']

print(f"   Buy-and-Hold Max Drawdown: {bh_drawdown:.1%}")
print(f"   Buy-and-Hold Volatility: {bh_vol:.1%}")

safer_strategies = comparison_df[
    (comparison_df['max_drawdown'] < bh_drawdown) & 
    (comparison_df.index != 'Buy-and-Hold')
]

if len(safer_strategies) > 0:
    print(f"\n   ✅ {len(safer_strategies)} strategies had smaller drawdowns")
    safest = safer_strategies['max_drawdown'].idxmin()
    reduction = ((bh_drawdown - safer_strategies.loc[safest, 'max_drawdown']) / bh_drawdown)
    print(f"   Safest: {safest}")
    print(f"   Drawdown reduced by: {reduction:.1%}")
else:
    print("\n   ⚠️  No strategy reduced drawdown vs buy-and-hold")

# 5. Trade-off analysis
print("\n\n5️⃣  THE TRADE-OFF:\n")

print("   When trimming helped:")
print("   - Locking in gains during market peaks")
print("   - Reducing exposure to individual stock risk")
print("   - Psychologically easier (selling winners feels good)")

print("\n   When trimming hurt:")
print("   - Sold winners that kept rising (opportunity cost)")
print("   - Trading fees reduced returns")
print("   - Cash drag (especially with 'cash' reinvestment mode)")

print("\n" + "="*80)
print("END OF ANALYSIS")
print("="*80)

---
## Export Results

### Generate Validator-Compatible Files

Let's export detailed results in a format that can be independently validated:

In [ ]:
# Create results directory
import os
import json

results_dir = 'results'
os.makedirs(results_dir, exist_ok=True)
print(f"📁 Created results directory: {results_dir}/\n")

# Store strategy objects for export
strategy_objects = {}

# Re-run strategies with tracking for export
print("=" * 80)
print("GENERATING VALIDATOR-COMPATIBLE EXPORT FILES")
print("=" * 80)

# Export Buy-and-Hold
print("\n🔄 Exporting Buy-and-Hold...")
strategy_name_clean = "buy_and_hold"

# 1. Portfolio value over time
pf_value = bh_portfolio.value()
pf_shares = pd.DataFrame(
    data=0.0,
    index=price_data.index,
    columns=price_data.columns
)
for ticker, shares in initial_shares.items():
    pf_shares.loc[:, ticker] = shares

portfolio_value_df = pf_shares.copy()
portfolio_value_df['Cash'] = 0.0  # Buy-and-hold keeps no cash
portfolio_value_df['Total_Value'] = pf_value

portfolio_value_df.to_csv(f"{results_dir}/{strategy_name_clean}_portfolio_value.csv")
print(f"  ✓ {strategy_name_clean}_portfolio_value.csv")

# 2. Metrics
metrics_df = pd.DataFrame([bh_stats])
metrics_df.to_csv(f"{results_dir}/{strategy_name_clean}_metrics.csv", index=False)
print(f"  ✓ {strategy_name_clean}_metrics.csv")

# 3. Trades (empty for buy-and-hold)
trades_df = pd.DataFrame(columns=['date', 'ticker', 'shares_sold', 'price', 'proceeds', 'gain_pct'])
trades_df.to_csv(f"{results_dir}/{strategy_name_clean}_trades.csv", index=False)
print(f"  ✓ {strategy_name_clean}_trades.csv")

# 4. Weights over time
weights_df = pf_shares.copy()
for col in weights_df.columns:
    weights_df[col] = (weights_df[col] * price_data[col]) / portfolio_value_df['Total_Value']
weights_df.to_csv(f"{results_dir}/{strategy_name_clean}_weights.csv")
print(f"  ✓ {strategy_name_clean}_weights.csv")

# 5. Metadata
metadata = {
    'strategy_name': 'Buy-and-Hold',
    'initial_capital': INITIAL_CASH,
    'start_date': START_DATE,
    'end_date': END_DATE,
    'tickers': list(price_data.columns),
    'trim_threshold': None,
    'trim_percentage': None,
    'reinvest_mode': None,
    'fees': 0.001
}
with open(f"{results_dir}/{strategy_name_clean}_metadata.json", 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"  ✓ {strategy_name_clean}_metadata.json")

# Export all trim strategies
for threshold in TRIM_THRESHOLDS:
    for mode in REINVEST_MODES:
        strategy_name = f"Trim@+{int(threshold*100)}% ({mode})"
        strategy_name_clean = f"trim_{int(threshold*100)}pct_{mode}"

        print(f"\n🔄 Exporting {strategy_name}...")

        # Re-run strategy with tracking
        strategy = TrimStrategy(
            price_data=price_data,
            initial_shares=initial_shares,
            threshold=threshold,
            trim_pct=TRIM_PERCENTAGE,
            reinvest_mode=mode
        )

        holdings_history = strategy.run_backtest()

        # Create portfolio for metrics
        portfolio = vbt.Portfolio.from_orders(
            close=price_data,
            size=holdings_history,
            size_type='targetamount',
            init_cash=INITIAL_CASH,
            fees=0.001,
            freq='D',
            group_by=True
        )

        # 1. Portfolio value over time
        portfolio_value_df = holdings_history.copy()
        portfolio_value_df['Cash'] = strategy.cash
        portfolio_value_df['Total_Value'] = portfolio.value()
        portfolio_value_df.to_csv(f"{results_dir}/{strategy_name_clean}_portfolio_value.csv")
        print(f"  ✓ {strategy_name_clean}_portfolio_value.csv")

        # 2. Metrics
        stats = {
            'final_value': portfolio.final_value,
            'total_return': portfolio.total_return,
            'cagr': portfolio.annualized_return,
            'sharpe_ratio': portfolio.sharpe_ratio,
            'sortino_ratio': portfolio.sortino_ratio,
            'max_drawdown': portfolio.max_drawdown,
            'volatility': portfolio.annualized_volatility,
            'num_trades': len(strategy.trim_log),
            'cash_held': strategy.cash
        }
        metrics_df = pd.DataFrame([stats])
        metrics_df.to_csv(f"{results_dir}/{strategy_name_clean}_metrics.csv", index=False)
        print(f"  ✓ {strategy_name_clean}_metrics.csv")

        # 3. Trades
        trades_df = pd.DataFrame(strategy.trim_log)
        if len(trades_df) > 0:
            trades_df['date'] = pd.to_datetime(trades_df['date'])
        trades_df.to_csv(f"{results_dir}/{strategy_name_clean}_trades.csv", index=False)
        print(f"  ✓ {strategy_name_clean}_trades.csv ({len(trades_df)} trades)")

        # 4. Weights over time
        weights_df = holdings_history.copy()
        total_position_value = sum(holdings_history[col] * price_data[col] for col in holdings_history.columns)
        for col in weights_df.columns:
            weights_df[col] = (weights_df[col] * price_data[col]) / portfolio_value_df['Total_Value']
        weights_df.to_csv(f"{results_dir}/{strategy_name_clean}_weights.csv")
        print(f"  ✓ {strategy_name_clean}_weights.csv")

        # 5. Metadata
        metadata = {
            'strategy_name': strategy_name,
            'initial_capital': INITIAL_CASH,
            'start_date': START_DATE,
            'end_date': END_DATE,
            'tickers': list(price_data.columns),
            'trim_threshold': threshold,
            'trim_percentage': TRIM_PERCENTAGE,
            'reinvest_mode': mode,
            'fees': 0.001
        }
        with open(f"{results_dir}/{strategy_name_clean}_metadata.json", 'w') as f:
            json.dump(metadata, f, indent=2)
        print(f"  ✓ {strategy_name_clean}_metadata.json")

print("\n" + "=" * 80)
print("✅ ALL EXPORT FILES GENERATED")
print("=" * 80)
print(f"\nFiles saved to: {results_dir}/")
print(f"Total strategies exported: {len(all_results)}")
print("\n🔍 Ready for independent validation!")

In [ ]:
# Save comparison results to CSV for further analysis
output_file = 'trimming_strategy_results.csv'
comparison_df.to_csv(output_file)
print(f"\n✅ Summary results exported to: {output_file}")

# Summary statistics
print("\n📊 Session Summary:")
print(f"   Strategies tested: {len(all_results)}")
print(f"   Tickers analyzed: {len(price_data.columns)}")
print(f"   Backtest period: {START_DATE} to {END_DATE}")
print(f"   Trading days: {len(price_data):,}")
print(f"\n   Best strategy: {comparison_df.index[0]}")
print(f"   Best final value: ${comparison_df.iloc[0]['final_value']:,.0f}")
print("\n✨ Analysis complete!")